In [1]:
# General import and load data
import pandas as pd
import numpy as np
import os
import re
from sklearn.pipeline import make_pipeline
from sklearn.svm import SVC
from sklearn.model_selection import learning_curve

from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler 
from sklearn.decomposition import PCA

from pandas import Series, DataFrame

# Training and test spliting
from sklearn.model_selection import train_test_split
from sklearn import preprocessing

# Estimators
from sklearn import tree

# Evaluation
from sklearn import metrics
from sklearn.model_selection import cross_val_score, KFold, StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score

# Optimization
from sklearn.model_selection import GridSearchCV

# Visualisation
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(color_codes=True)


# if matplotlib is not set inline, you will not see plots
#alternatives auto gtk gtk2 inline osx qt qt5 wx tk
#%matplotlib auto
#%matplotlib qt
%matplotlib inline

url_df = 'https://raw.githubusercontent.com/AngelaBurgaleta/TFM_NHANES/main/DATASETS%20DEFINITIVOS/PROCESSED/df_procesado_genderandage.csv'
url_df45_sinoutliers = 'https://raw.githubusercontent.com/AngelaBurgaleta/TFM_NHANES/main/DATASETS%20DEFINITIVOS/PROCESSED/Sin_outliers/df_v45sinoutliers.csv'
df = pd.read_csv(url_df45_sinoutliers)
df = df.drop(['Unnamed: 0'], axis = 1)
#Tutotial: https://www.kaggle.com/code/prashant111/svm-classifier-tutorial/notebook

In [2]:
y = df['MET_SYM']
vars_descarte = ['REQ1', 'REQ2', 'REQ3', 'REQ4', 'REQ5', 'REQ_SUM', 'MET_SYM', 'BPXSY1', 'BPXDI1', 'BMXWAIST', 'LBXTR', 'BPQ090D', 'LBDHDD', 'BPQ040A', 'LBXGLU', 'DIQ070', 'BMXHT', 'BMXWT', 'SEQN', 'DPQ050', 'MCQ010', 'DBQ700', 'ALQ130', 'ALQ130_Tramos', 'PAQ665', 'PAQ620', 'PAQ605', 'BPQ050A']
X = df.drop(vars_descarte, axis = 1)

vars_descarte_ms = ['REQ1', 'REQ2', 'REQ3', 'REQ4', 'REQ5', 'REQ_SUM', 'BPXSY1', 'BPXDI1', 'BMXWAIST', 'LBXTR', 'BPQ090D', 'LBDHDD', 'BPQ040A', 'LBXGLU', 'DIQ070', 'BMXHT', 'BMXWT', 'SEQN', 'DPQ050', 'MCQ010', 'DBQ700', 'ALQ130', 'ALQ130_Tramos', 'PAQ665', 'PAQ620', 'PAQ605', 'BPQ050A']
df_ms = df.drop(vars_descarte_ms, axis = 1)



In [3]:
#SIN PCA
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=30)

In [ ]:
#CON PCA
#Normalización
pca_pipe = make_pipeline(StandardScaler(), PCA())
pca_pipe.fit(X)
# Se extrae el modelo entrenado del pipeline
modelo_pca = pca_pipe.named_steps['pca']
#Aplicar PCA 
#fijamos componentes
pca = PCA(n_components=26)
X_pca = pca.fit_transform(X)
X_pca.shape
#Separar datos en train, test, entrada y salida despues del PCA
X_train_pca, X_test_pca, y_train, y_test = train_test_split(X_pca, y, test_size=0.2, random_state=30)

##Definir modelo simple

In [4]:

types_of_kernels = ['linear', 'rbf', 'poly']

kernel = types_of_kernels[0]
gamma = 3.0

# Create SVC model
model = SVC(kernel=kernel, probability=True, gamma=gamma)

##Train and evaluate

In [6]:
#This step will take some time 
# Train - This is not needed if you use K-Fold

model.fit(X_train, y_train)

predicted = model.predict(X_test)
expected = y_test

NameError: ignored

In [ ]:
# Accuracy
metrics.accuracy_score(expected, predicted)
#results depend on the splitting

##Null accuracy

In [ ]:
# Count number of samples per class
s_y_test = Series(y_test)
s_y_test.value_counts()

In [ ]:
# Mean of ones
y_test.mean()

In [ ]:
# Mean of zeros
1 - y_test.mean() 

In [ ]:
# Calculate null accuracy (binary classification coded as 0/1)
max(y_test.mean(), 1 - y_test.mean())

In [ ]:
# Calculate null accuracy (multiclass classification)
s_y_test.value_counts().head(1) / len(y_test)

##Confussion matrix and F score

We can obtain more information from the confussion matrix and the metric F1-score.
In a confussion matrix, we can see:

||**Predicted**: 0| **Predicted: 1**|
|---------------------------|
|**Actual: 0**| TN | FP |
|**Actual: 1**| FN|TP|

* **True negatives (TN)**: actual negatives that were predicted as negatives
* **False positives (FP)**: actual negatives that were predicted as positives
* **False negatives (TN)**: actual positives that were predicted as negatives
* **True negatives (TN)**: actual positives that were predicted as posiives

We can calculate several metrics from the confussion matrix

* **Recall** (also called *sensitivity*): when the actual value is positive, how often the prediction is correct? 
(TP / (TP + FN))
* **Specificity**: when the actual value is negative, how often the prediction is correct? (TN / (TN + FP))
* **False Positive Rate**: when the actual value is negative, how often the prediction is incorrect? (FP / (TN + FP))
* **Precision**: when a positive value is predicted, how many times is correct? (TP / (TP + FP)
A good metric is F1-score: 2TP / (2TP + FP + FN)

In [ ]:
# Confusion matrix
print(metrics.confusion_matrix(expected, predicted))

In [ ]:
# Report
print(classification_report(expected, predicted))

##ROC and AUC

The [ROC](https://en.wikipedia.org/wiki/Receiver_operating_characteristic)  curve illustrates the performance of a binary classifier system as its discrimination threshold is varied.

In [ ]:
y_pred_prob = model.predict_proba(X_test_pca)[:,1]
fpr, tpr, thresholds = roc_curve(y_test, y_pred_prob)
plt.plot(fpr, tpr)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.0])
plt.title('ROC curve for Titanic')
plt.xlabel('False Positive Rate (1 - Recall)')
plt.xlabel('True Positive Rate (Sensitivity)')
plt.grid(True)

In [ ]:
#Threshold used by the decision function, thresholds[0] is the number of 
thresholds

In [ ]:
#Histogram of probability vs actual
dprob = pd.DataFrame(data = {'probability':y_pred_prob, 'actual':y_test})
dprob.probability.hist(by=dprob.actual, sharex=True, sharey=True)

In [ ]:
#Function to evaluate thresholds of the ROC curve
def evaluate_threshold(threshold):
    print('Sensitivity:', tpr[thresholds > threshold][-1])
    print('Recall:', 1 - fpr[thresholds > threshold][-1])

In [ ]:
evaluate_threshold(0.74)

In [ ]:
evaluate_threshold(0.5)

###Comments
By default, the thresdhold to decide a class is 0.5, If we modify it, we should use the new thresdhold.

threshold = 0.8

predicted = model.predict_proba(X) > threshold

AUC is the percentage of the ROC plot underneath the curve. Represents the likelihood that the predictor assigns  a higher predicted probability to the positive observation.  A simple rule  to evaluate a classifier based on this summary value is the following:
* .90-1 = very good (A)
* .80-.90 = good (B)
* .70-.80 = not so good (C)
* .60-.70 = poor (D)
* .50-.60 = fail (F)

In [ ]:
# AUX
print(roc_auc_score(expected, predicted))

##Train and Evaluate with K-fold

This is alternative to splitting the dataset into train and test. It will run k times slower than the other method, but it will be more accurate.

In [ ]:
# This step will take some time
# Cross-validationt
cv = KFold(n_splits=5, shuffle=False, random_state=33)
# StratifiedKFold has is a variation of k-fold which returns stratified folds:
# each set contains approximately the same percentage of samples of each target class as the complete set.
#cv = StratifiedKFold(y, n_folds=3, shuffle=False, random_state=33)
scores = cross_val_score(model, X, y, cv=cv)
print("Scores in every iteration", scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))


In [ ]:
plot_learning_curve(model, "Learning curve with K-Fold", X_pca, y, cv=cv)

##Train and Optimize

In this section we are going to provide an alternative version of the previous one with optimization

In [ ]:
#Tune parameters
gammas = np.logspace(-6, -1, 10)
gs = GridSearchCV(model, param_grid=dict(gamma=gammas))
gs.fit(X_train, y_train)
scores = gs.score(X_test, y_test)
print(scores)

In [ ]:
# Refine model
model = SVC(kernel='linear', gamma=gs.best_estimator_.gamma)
plot_learning_curve(model, "optimized with GridSearch", X , y, cv=cv)

#ROC

In [ ]:


y_test_pred = model.predict(X_test)

In [ ]:
from sklearn.metrics import roc_curve, roc_auc_score

In [ ]:
clf_tree = model

In [ ]:
y_score1 = clf_tree.predict_proba(X_test)[:,1]

In [ ]:
false_positive_rate1, true_positive_rate1, threshold1 = roc_curve(y_test, y_score1)

In [ ]:
print('roc_auc_score for DecisionTree: ', roc_auc_score(y_test, y_score1))print('roc_auc_score for DecisionTree: ', roc_auc_score(y_test, y_score1))

In [ ]:
plt.subplots(1, figsize=(10,10))
plt.title('Receiver Operating Characteristic - DecisionTree')
plt.plot(false_positive_rate1, true_positive_rate1)
plt.plot([0, 1], ls="--")
plt.plot([0, 0], [1, 0] , c=".7"), plt.plot([1, 1] , c=".7")
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()


In [ ]:
print('roc_auc_score for DecisionTree: ', roc_auc_score(y_test, y_score1))

##Overfitting


In [ ]:
# print the scores on training and test set

print('Training set score: {:.4f}'.format(linear_svc.score(X_train, y_train)))

print('Test set score: {:.4f}'.format(linear_svc.score(X_test, y_test)))